# Ipython Canvas Tool Embedding Experiment

Break away from using tkinter canvas and see if the same functionality can be created using only Ipython widgets.

## Research

### ipycanvas
https://github.com/martinRenou/ipycanvas
This tool came up earlier in research phase. Unused because the canvas drawing style was discontinuous and dot-like, making it look less like smooth handwriting.

### list of widgets
https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

### alternatives to tkinter canvas
https://wiki.tcl-lang.org/page/Alternative+Canvases


# Ipycanvas (Revisitied)

Originally found in **experiments/Testing ipycanvas**. We can recreate the canvas code here and see if there are changes we can adapt to make it as deployable as the tkinter prototype.

**How does this compare to the tkinter code?**

- **tkinter:** 
    - fast
    - is not embedded: b/c it uses its own way of poppinp up a window
    - binds command to button 
    - instantiates tkinter canvas   

- **ipycanvas:**
    - slow
        - tends to be marginally slower than the tkinter canvas
    - is embedded
    - roughcanvas 
    - uses roughcanvas.RoughCanvas 
    - not sure if it can support other functionality as tkinter's canvas such as button saving 

**Development**

This tool is working out okay, despite any cons. It is capable of embedding a can as immediately in a cell.  
Functionality is being added to send this canvas for prediction using previously built code.

In [1]:
import os
import base64
from pathlib import Path

import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
from ipywidgets import Image
from ipywidgets import ColorPicker, IntSlider, link, AppLayout, HBox, VBox

from ipycanvas import RoughCanvas, hold_canvas
from ipyfilechooser import FileChooser

import PIL
from PIL import Image, ImageDraw

from IPython.display import Markdown as md

import requests
import json

import configs as cfg

In [2]:
?display

Signature:
display(
    *objs,
    include=None,
    exclude=None,
    metadata=None,
    transient=None,
    display_id=None,
    **kwargs,
)
Docstring:
Display a Python object in all frontends.

By default all representations will be computed and sent to the frontends.
Frontends can decide which representation is used and how.

In terminal IPython this will be similar to using :func:`print`, for use in richer
frontends see Jupyter notebook examples with rich display logic.

Parameters
----------
objs : tuple of objects
    The Python objects to display.
raw : bool, optional
    Are the objects to be displayed already mimetype-keyed dicts of raw display data,
    or Python objects that need to be formatted before display? [default: False]
include : list, tuple or set, optional
    A list of format type strings (MIME types) to include in the
    format data dict. If this is set *only* the format types included
    in this list will be computed.
exclude : list, tuple or set, optional
  

In [3]:
# Mathpix API key details.
app_id = cfg.math_pix_key["app_id"]  
app_key = cfg.math_pix_key["app_key"]

In [4]:
def ocr_request(filename):
    """Sends an API request to MathPix API to be handled by the OCR .
    
    :param filename: Filename of the image to be sent to the MathPix API for a LaTeX prediction.
    :type filename: str
 
    :return: Latex-formatted string.
    :rtype: str
    """
    
    dict_request={
            "src": "data:image/png",
            "formats": ["text", "data", "html"],
            "data_options": {
            "include_asciimath": True,
            "include_latex": True
            }
        }

    # Put desired filename from earlier.
    image_uri = "data:image/png;base64," + base64.b64encode(open(filename, "rb").read()).decode()

    # Send the request.
    r = requests.post("https://api.mathpix.com/v3/text",
                      data=json.dumps({'src': image_uri}),
                      headers={"app_id": app_id, 
                               "app_key": app_key,
                               "Content-type": "application/json"})
    
    json_return = json.loads(r.text)
    latex_return = json_return.get("latex_styled")
    
    return latex_return

In [5]:
def refresh_file_browse(change):
    filename = fc.children[1].children[0].value + "/" + change.new
    file = open(filename, "rb")
    image = file.read()
    img_preview.value = image

def img_preview_handler(change):
    print('this')
#         prev_box.children[0].value = image
#         print("refresh_file_browse")
#         img_preview.value = image

def btn_save_eventhandler(self):
    filename["name"] = "tempfile.jpg"
    canvas_image.save(filename["name"])  
    print("file saved as: ", filename["name"])
    
def btn_pred_eventhandler(self):
    filename["name"] = "tempfile.jpg"
    canvas_image.save(filename["name"]) 
    
    if fc.selected is not None:
        latex_return = ocr_request(fc.selected)
    else:
        latex_return = ocr_request(filename["name"])
        
    if latex_return is not None:
        latex_pred[0] = latex_return
        txt_preview.value =  latex_pred[0]
        
def on_mouse_down(x, y):
    global drawing
    global position
    global shape

    drawing = True
    position = (x, y)
    shape = [position]
    
def on_mouse_move(x, y):
    global drawing
    global position
    global shape

    x_0 = position[0]
    y_0 = position[1]

    if not drawing:
        return

    with hold_canvas(canvas):
        canvas.stroke_line(x_0, y_0, x, y)
        position = (x, y)
    shape.append(position)

def on_mouse_up(x, y, canvas_fill=False):
    global drawing
    global position
    global shape
    
    x_0 = position[0]
    y_0 = position[1]
    
    drawing = False
    
    with hold_canvas(canvas):
        canvas.stroke_line(x_0-1, y_0-1, x+1, y+1)
        
    for i in range(len(shape)):
        x_0 = shape[i][0]
        y_0 = shape[i][1]
        x_1 = shape[i+1][0]
        y_1 = shape[i+1][1]
        draw.line([x_0, y_0, x_1, y_1], fill="black", width=2)        
        
drawing = False
position = None
shape = []
width = 800
height = 400

latex_pred = ["No LaTeX Loaded."]
filename = {"name":"tempfile.jpg"}
init_browse_dir = os.getcwd()

# Instantiate ipycanvas RoughCanvas and use 
# PIL to create an empty image and draw object to draw on
# memory only, not visible to the user.
canvas = RoughCanvas(width=width, height=height)
canvas_image = PIL.Image.new("L", (width, height), color="white")
draw = ImageDraw.Draw(canvas_image)

# Canvas actions.
canvas.on_mouse_down(on_mouse_down)
canvas.on_mouse_move(on_mouse_move)
canvas.on_mouse_up(on_mouse_up)
canvas.stroke_style = "black"
canvas.line_width = 1.5

# File browser widget.
init_browse_dir = str(Path.home())
fc = FileChooser(init_browse_dir, select_desc="Select")
fc.use_dir_icons = True
fc.filter_pattern = ['*.jpg', '*.png']

# Image preview widget.
file = open(filename["name"], "rb")
image = file.read()
img_preview = widgets.Image(
    value=image,
    format='png',
    width=600,
    height=300,
)

# Text preview widget.
txt_preview = widgets.Text(
    value=latex_pred[0],
    placeholder='Type something',
    description='String:',
    disabled=False
)

# Buttons.
# btn_browse = widgets.Button(description="Browse Files")
btn_save = widgets.Button(description="Save Canvas")
btn_pred = widgets.Button(description="Predict LaTeX!")

# btn_browse.on_click(btn_browse_eventhandler)
btn_save.on_click(btn_save_eventhandler)
btn_pred.on_click(btn_pred_eventhandler)

# Layout.
box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='inherit',
                    border='solid',
                    width='100%')

box_prev_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='inherit',
                    border='solid',
                    width='100%')

box_pred_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='center',
                    border='solid',
                    width='100%')

box_canv_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='inherit',
                    border='solid',
                    width='100%')
# Boxes.
btn_box = Box(children=[fc, btn_save, btn_pred], layout=box_layout)
prev_box = Box(children=[img_preview], layout=box_prev_layout)
pred_box = Box(children=[txt_preview], layout=box_pred_layout)
canv_box =  Box(children=[canvas], layout=box_canv_layout)

# Observers.
fc.children[1].children[1].observe(refresh_file_browse, names='value')
img_preview.observe(img_preview_handler, names='value')

In [684]:
# print(fc.children[1].children[0].value)

/Users/jaimelopez


In [6]:
vb = VBox((btn_box, prev_box, pred_box, canv_box,))
display(vb)

In [7]:
print(filename["name"])
print(filename["name"])
prev_box.children[0]

tempfile.jpg
tempfile.jpg


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x0…

In [647]:
# Text preview widget.
def this_handler(change):
    print("this handler success")

def common_filtering(year, purpose):
    """
    Takes two input values and applies both values as filters
    to the same dataframe.
    
    year = string.
    purpose = string.
    """
    ALL = 'ALL'
    
    output.clear_output()
    
    if (year == ALL) and (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[
            (df_london.year == year) 
            & (df_london.purpose == purpose)]

# File browser widget.
init_browse_dir = str(Path.home())
fc = FileChooser(init_browse_dir, select_desc="File Browse")
fc.use_dir_icons = True
fc.filter_pattern = ['*.jpg', '*.png']


txt_preview = widgets.Text(
    value=latex_pred[0],
    placeholder='Type something',
    description='String:',
    disabled=False
)

txt_preview.observe(this_handler, names='value')
txt_preview

fc.children[1].children[1].observe(this_handler, names='value')
fc.children[1].children[1].keys
fc


FileChooser(path='/Users/jaimelopez', filename='', title='HTML(value='', layout=Layout(display='none'))', show…

In [648]:
# print(fc.children[1])
print(fc.children[1].children[1].value)

### Post production work

In [320]:
# md("$ %s $"%(latex_pred))

# latex_string = f"$$\huge {latex_pred[0]}$$"

# md(latex_string)

In [321]:
latex_pred[0]

'No LaTeX Loaded.'